# Media

## Introduction


**Scikit-rf** supports s-parameter simulations of transmission line models such as hollow waveguide, coaxial, microstripline and free air among others.

The [Media](../api/media/index.rst) object store the properties of a media, such as `propagation constant` and `characteristic impedance`. The networks objects - like a line of a given length - are generated by the [Media](../api/media/index.rst).

This tutorial illustrates how to create networks using several kind of [Media](../api/media/index.rst). It also explains the most common pitfalls regarding the network port and characteristic impedances.

The singular of the latin word *media* is *medium*, which means middle or centre. For clarity sake, *media* is used for both singular and plural in **scikit-rf**.

## Basic example

Let's say we want to create two networks, a length of 2-conductor 50 ohm planar microstripline and a length of 1-conductor WR-10 rectangular waveguide.

| Microstripline | Rectangular Waveguide |
| :-: | :-: |
| ![microstripline](figures/media_mline.svg) | ![rectangular waveguide](figures/media_rectangularwaveguide.svg) |

First of all, two [Media](../api/media/index.rst) objects must be created with the models parameters.

### Media Parameters

| Microstripline | | | WR-10 Rectangular Waveguide | | |
| :- | :- | :- | :- | :- | :- |
| Track width | `w` | 3 mm | Aperture width | `a` | 100 mil |
| Track thickness | `t` |  35 um| Aperture height | `b` | 50 mil |
| Substrate height | `h` | 1.6 mm | - | - | - |
| Relative permittivity (FR-4) | `ep_r` | 4.5  | Relative permittivity (air) | `ep_r`| 1.0 |
| Resistivity (copper) | `rho` | 1.68e-08 ohm * m  |  Resistivity (copper) | `rho` | 1.68e-08 ohm * m |

In [ ]:
# various initialization
%matplotlib inline
import skrf as rf
import matplotlib.pyplot as plt
rf.stylely()

# import the desired media and the frequency axis
from skrf.media import MLine
from skrf.media import RectangularWaveguide
from skrf import Frequency
from skrf.constants import to_meters

# create frequency axes
f_mlin = Frequency(0.1,10,101, 'Ghz')
f_wr10 = Frequency(75, 110, 101,'Ghz')

# create media from parameters
mlin  =  MLine(f_mlin, w=3e-3, h=1.6e-3, t=35e-6, ep_r=4.5,  rho=1.68e-08)
print(mline)
wr10  = RectangularWaveguide(f_wr10, a=to_meters(100, 'mil'), b=to_meters(50, 'mil'), ep_r=1.0,  rho=1.68e-08)
print(wr10)

### Line creation

Secondly, the [Media](../api/media/index.rst) objects are used to generate networks corresponding to a 100 millimeter length of both media.

In [ ]:
# create the transmission line networks
mlin_100 = mlin.line(100e-3, name = 'mlin 100mm')
print(mlin_100)
wr10_100 = wr10.line(100e-3, name = 'wr10 100mm')
print(wr10_100)

### Plotting the results

The S-Parameters of the lines can be plotted:

In [ ]:
# prepare figure
fig1, axes = plt.subplots(2, 2, figsize = (8, 5))

# plot miscrostipline
mlin_100.plot_s_mag(0, 0, ax=axes[0,0])
mlin_100.plot_s_db(1, 0, ax=axes[0,1])

# plot rectangular waveguide
wr10_100.plot_s_mag(0, 0, ax=axes[1,0])
wr10_100.plot_s_db(1, 0, ax=axes[1,1])

# resize plot nicely
axes[0, 0].set_ylim((-1, 1))
axes[1, 0].set_ylim((-1, 1))
fig1.tight_layout()

The insertion loss S21 of the transmission line is frequency dependant, but the S11 magnitude is constant and zero. Absolute magnitude of S11 was ploted instead of dB because log(0) is undefined. 

Why does return loss S11 not show the shape with maxima and valleys due to length as observed on vector network analyser measurements? 

This is because no port impedance `z0` was specified at media construction. The characteristic impedance `Z0` was used as port impedance and the network is perfectly matched with itself. This result is equivalent to an electromagnetic simulation with complex port impedance instead of a real-world measurement.

![simulation](figures/media_simulation.svg)

The impedances port and characteristic impedances can be plotted and are frequency dependant and equals:

In [ ]:
# prepare figure
fig2, axes = plt.subplots(1, 2, figsize = (8, 5))

# plot miscrostipline
axes[0].plot(mlin_100.frequency.f_scaled, mlin_100.z0[:, 0].real, marker = '.', label = 'mlin 100mm z0')
axes[0].plot(mlin.frequency.f_scaled, mlin.Z0.real, label = 'mlin Z0')
axes[0].set_ylabel('Impedance (ohm)')
axes[0].set_xlabel('Frequency (GHz)')
axes[0].set_title('Microstripline')
axes[0].legend()

# plot rectangular waveguide
axes[1].plot(wr10_100.frequency.f_scaled, wr10_100.z0[:, 0].real, marker = '.', label = 'wr10 100mm z0')
axes[1].plot(wr10.frequency.f_scaled, wr10.Z0.real, label = 'wr10 Z0')
axes[1].set_ylabel('Impedance (ohm)')
axes[1].set_xlabel('Frequency (GHz)')
axes[1].set_title('WR-10')
axes[1].legend()

# resize plot nicely
fig1.tight_layout()

### Measured-like microstripline

Measuring s-parameters of 2-conductor microstriplines is done with a VNA (vector network analyser). The VNA has a known port impedance - usually 50 ohm - and coaxial connectivity. A coaxial to microstripline transition has to be used. In this example let's assume an ideal transition (no length, perfect match).

![mline measurement](figures/media_mline_measurement.svg)




### Measured-like WR-10 waveguide

![waveguide measurement](figures/media_waveguide_measurement.svg)

More detailed examples illustrating how to create various kinds of [Media](../api/media/index.rst) 
objects are given below. A full list of media's supported can be found in the  [Media](../api/media/index.rst) API page. The network creation and connection syntax of **skrf** are cumbersome  if you need to doing complex circuit design. **skrf**'s synthesis capabilities lend themselves more to scripted applications such as calibration, optimization or batch processing.

##  [Media](../api/media/index.rst) Object Basics 

Two arguments are common to all media constructors

* `frequency` (required)
*  `z0`   (optional)

`frequency` is  a `Frequency` object, and `z0` is the port impedance. `z0` is only needed if the port impedance is different from the media's characteristic impedance. Here is an example of how to initialize a coplanar waveguide [0] media. The instance has  a 10um center conductor, gap of 5um, and substrate with relative permativity of 10.6,

In [ ]:
from skrf.media import CPW
freq = Frequency(75,110,101,'ghz')
cpw = CPW(freq, w=10e-6, s=5e-6, ep_r=10.6, z0 =1)

cpw

For the purpose of microwave network analysis, the defining properties of a (single moded) transmission line are it's characteristic impedance and propagation constant. These properties return complex `numpy.ndarray`'s,  A *port impedance* is also needed when different networks are connected. 

The *characteristic impedance* is given by a `Z0` (capital Z)

In [ ]:
cpw.Z0[:3] 

The *port impedance* is given by `z0` (lower z).  Which we set to 1, just to illustrate how this works. The *port impedance* is used to compute impedance mismatched if circuits of different port impedance are connected. 

In [ ]:
cpw.z0[:3]

The propagation constant is given by `gamma` 

In [ ]:
cpw.gamma[:3]

Lets take a look at some other [Media](../api/media/index.rst)'s

## Slab of  Si in Freespace

A plane-wave in freespace from 10-20GHz.

In [ ]:
from skrf.media import Freespace

freq = Frequency(10,20,101,'ghz')
air =  Freespace(freq)
air

In [ ]:
air.z0[:2] # 377ohm baby!

In [ ]:
# plane wave in Si
si = Freespace(freq, ep_r = 11.2)
si.z0[:3] # ~110ohm


Simulate a 1cm slab of Si in half-space,

In [ ]:
slab = air.thru() ** si.line(1, 'cm') ** air.thru()
slab.plot_s_db(n=0)

## Rectangular Waveguide

a WR-10 Rectangular Waveguide

In [ ]:
from skrf.media import RectangularWaveguide

freq = Frequency(75,110,101,'ghz')
wg = RectangularWaveguide(freq, a=100*rf.mil, z0=50) # see note below about z0
wg

The `z0` argument in the Rectangular Waveguide constructor is used
to force a specific port impedance. This is commonly used to match 
the port impedance to what a VNA stores in a touchstone file. Lets compare the propagation constant in waveguide to that of freespace, 

In [ ]:
air = Freespace(freq)

In [ ]:
from matplotlib import pyplot as plt

air.plot(air.gamma.imag, label='Freespace')
wg.plot(wg.gamma.imag, label='WR10')

plt.ylabel('Propagation Constant (rad/m)')
plt.legend()

Because the wave quantities are dynamic they change when the attributes 
of the media change. To illustrate, plot the propagation constant of the cpw for various values of substrated permativity,  

In [ ]:
for ep_r in [9,10,11]:
    cpw = CPW(freq, w=10e-6, s=5e-6, ep_r=ep_r, z0 =1)
    cpw.frequency.plot(cpw.beta, label='er=%.1f'%ep_r)

plt.xlabel('Frequency [GHz]')
plt.ylabel('Propagation Constant [rad/m]')
plt.legend()


## Network Synthesis

Networks are created through methods of a Media object. To create a 1-port network for a rectangular waveguide short, 

In [ ]:
wg.short(name = 'short') 

Or to create a $90^{\circ}$ section of cpw line, 

In [ ]:
cpw.line(d=90,unit='deg', name='line')

## Building Circuits


By connecting a series of simple circuits, more complex circuits can be 
made. To build a the $90^{\circ}$ delay short, in the 
rectangular waveguide media defined above.

In [ ]:
delay_short = wg.line(d=90,unit='deg') ** wg.short()
delay_short.name = 'delay short'
delay_short

When `Networks` with more than 2 ports need to be connected together, use 
`rf.connect()`.  To create a two-port network for a shunted delayed open, you can create an ideal 3-way splitter (a 'tee') and connect the delayed open to one of its ports,
	

In [ ]:
tee = cpw.tee()
delay_open = cpw.delay_open(40,'deg')
shunt_open = rf.connect(tee,1,delay_open,0)

Adding networks in shunt  is pretty common, so there is a `Media.shunt()` function to do this, 

In [ ]:
cpw.shunt(delay_open)

If a specific circuit is created frequently, it may make sense to 
use a function to create the circuit. This can be done most quickly using `lambda`

In [ ]:
delay_short = lambda d: wg.line(d,'deg')**wg.short()
delay_short(90)

A more useful example may be to create a function for a shunt-stub tuner,
that will work for any media object

In [ ]:
def shunt_stub(med, d0, d1):
    return med.line(d0,'deg')**med.shunt_delay_open(d1,'deg')

shunt_stub(cpw,10,90)

This approach lends itself to design optimization.

## Design Optimization


The abilities of `scipy`'s optimizers can be used to automate network design. In this example, skrf is used to automate the single stub impedance matching network design. First, we create a 'cost' function which returns something we want to minimize, such as the reflection coefficient magnitude at band center. Then, one of scipy's minimization algorithms is used to determine the optimal parameters of the stub lengths to minimize this cost.

In [ ]:
from scipy.optimize import fmin

# the load we are trying to match
load = cpw.load(.2+.2j)

# single stub circuit generator function
def shunt_stub(med, d0, d1):
    return med.line(d0,'deg')**med.shunt_delay_open(d1,'deg')


# define the cost function we want to minimize (this uses sloppy namespace)
def cost(d):
    # prevent negative length lines, returning high cost
    if d[0] <0 or d[1] <0:
        return 1e3 
    return (shunt_stub(cpw,d[0],d[1]) ** load)[100].s_mag.squeeze()

# initial guess of optimal delay lengths in degrees
d0 = 120,40 # initial guess

#determine the optimal delays
d_opt = fmin(cost,(120,40))

d_opt 